In [1]:
import os, sys
from os.path import dirname

root_path = os.path.abspath(os.getcwd())
if not root_path in sys.path: sys.path.insert(0, root_path)
    
print(sys.path)

['D:\\GitWork\\ants_and_bees', 'C:\\opt\\miniconda3\\envs\\py37\\python37.zip', 'C:\\opt\\miniconda3\\envs\\py37\\DLLs', 'C:\\opt\\miniconda3\\envs\\py37\\lib', 'C:\\opt\\miniconda3\\envs\\py37', '', 'C:\\opt\\miniconda3\\envs\\py37\\lib\\site-packages', 'C:\\opt\\miniconda3\\envs\\py37\\lib\\site-packages\\win32', 'C:\\opt\\miniconda3\\envs\\py37\\lib\\site-packages\\win32\\lib', 'C:\\opt\\miniconda3\\envs\\py37\\lib\\site-packages\\Pythonwin', 'C:\\opt\\miniconda3\\envs\\py37\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\Morpheus\\.ipython']


In [68]:
RootPath_Code = 'D:\\GitWork\\mnist\\'
RootPath_Data = 'D:\\GitWork\\mnist\\data\\'

Dir_Processed = 'processed'
Dir_Raw = 'raw'

Fname_Train = 'training.pt'
Fname_Test  = 'test.pt'

Resources = [
    "train-images-idx3-ubyte.gz", 
    "train-labels-idx1-ubyte.gz",
    "t10k-images-idx3-ubyte.gz",
    "t10k-labels-idx1-ubyte.gz"
]

path_processed = os.path.join(RootPath_Data, Dir_Processed)
path_raw = os.path.join(RootPath_Data, Dir_Raw)

if not os.path.exists(path_processed): os.makedirs(path_processed)

abspath_trainset = os.path.join(path_processed, Fname_Train)
abspath_testset  = os.path.join(path_processed, Fname_Test)

isTrainDataExist = os.path.exists(abspath_trainset)
isTestDataExist  = os.path.exists(abspath_testset)

print("'{}' exist: {}".format(abspath_trainset, isTrainDataExist))
print("'{}' exist: {}".format(abspath_testset, isTestDataExist))

isResourcesExist = [os.path.exists(os.path.join(path_raw, f)) for f in Resources]
print(isResourcesExist)

# If all resource files exist then return 'True' else 'False'
isAllResourcesExist = all(isResourcesExist)
print(isAllResourcesExist)

'D:\GitWork\mnist\data\processed\training.pt' exist: False
'D:\GitWork\mnist\data\processed\test.pt' exist: False
[True, True, True, True]
True


In [69]:
# If you want to download MNIST data by torchvision function

# your_data_path = ''
# torchvision.datasets.MNIST(your_data_path, train=True, transform=None, target_transform=None, download=True)

In [71]:
import codecs

def get_int(b):
    return int(codecs.encode(b, 'hex'), 16)

def open_maybe_compressed_file(path):
    """Return a file object that possibly decompresses 'path' on the fly.
       Decompression occurs when argument `path` is a string and ends with '.gz' or '.xz'.
    """
    if not isinstance(path, torch._six.string_classes):
        return path
    if path.endswith('.gz'):
        import gzip
        return gzip.open(path, 'rb')
    if path.endswith('.xz'):
        import lzma
        return lzma.open(path, 'rb')
    return open(path, 'rb')

def read_sn3_pascalvincent_tensor(path, strict=True):
    """Read a SN3 file in "Pascal Vincent" format (Lush file 'libidx/idx-io.lsh').
       Argument may be a filename, compressed filename, or file object.
    """
    # typemap
    if not hasattr(read_sn3_pascalvincent_tensor, 'typemap'):
        read_sn3_pascalvincent_tensor.typemap = {
            8: (torch.uint8, np.uint8, np.uint8),
            9: (torch.int8, np.int8, np.int8),
            11: (torch.int16, np.dtype('>i2'), 'i2'),
            12: (torch.int32, np.dtype('>i4'), 'i4'),
            13: (torch.float32, np.dtype('>f4'), 'f4'),
            14: (torch.float64, np.dtype('>f8'), 'f8')}
    # read
    with open_maybe_compressed_file(path) as f:
        data = f.read()
    # parse
    magic = get_int(data[0:4])
    nd = magic % 256
    ty = magic // 256
    assert nd >= 1 and nd <= 3
    assert ty >= 8 and ty <= 14
    m = read_sn3_pascalvincent_tensor.typemap[ty]
    s = [get_int(data[4 * (i + 1): 4 * (i + 2)]) for i in range(nd)]
    parsed = np.frombuffer(data, dtype=m[1], offset=(4 * (nd + 1)))
    assert parsed.shape[0] == np.prod(s) or not strict
    return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)

def read_image_file(path):
    with open(path, 'rb') as f:
        x = read_sn3_pascalvincent_tensor(f, strict=False)
    assert(x.dtype == torch.uint8)
    assert(x.ndimension() == 3)
    return x

def read_label_file(path):
    with open(path, 'rb') as f:
        x = read_sn3_pascalvincent_tensor(f, strict=False)
    assert(x.dtype == torch.uint8)
    assert(x.ndimension() == 1)
    return x.long()


def createProcessedData(resources, fname_files):

    training_set = (
        read_image_file(os.path.join(path_raw, resources[0])),
        read_label_file(os.path.join(path_raw, resources[1]))
    )

    test_set = (
        read_image_file(os.path.join(path_raw, resources[2])),
        read_label_file(os.path.join(path_raw, resources[3]))
    )
    
    with open(os.path.join(path_processed, fname_files[0]), 'wb') as f:
        torch.save(training_set, f)

    with open(os.path.join(path_processed, fname_files[1]), 'wb') as f:
        torch.save(test_set, f)
    

Fname_Files = [Fname_Train, Fname_Test]
createProcessedData(Resources, Fname_Files)

for f in data_fnames:
    isExist = os.path.exists(os.path.join(path_processed, f))
    print('{} exist: {}'.format(f, isTrainDataExist))


AssertionError: 

In [62]:
from __future__ import print_function, division  

import numpy as np
import time
import torch
import torchvision

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

100.1%

Extracting D:\Dataset\MNIST\raw\train-images-idx3-ubyte.gz to D:\Dataset\MNIST\raw


113.5%

Extracting D:\Dataset\MNIST\raw\train-labels-idx1-ubyte.gz to D:\Dataset\MNIST\raw


100.4%

Extracting D:\Dataset\MNIST\raw\t10k-images-idx3-ubyte.gz to D:\Dataset\MNIST\raw


180.4%

Extracting D:\Dataset\MNIST\raw\t10k-labels-idx1-ubyte.gz to D:\Dataset\MNIST\raw
Processing...
Done!


Dataset MNIST
    Number of datapoints: 60000
    Root location: D:\Dataset
    Split: Train